<a href="https://colab.research.google.com/github/rene-aum/Atlas/blob/main/Asignacion/nb1_asignacion_apartados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatización del proceso de Asignaciones
Por favor:
1. Genera la hoja de sheets de salida en la carpeta correspondiente: https://drive.google.com/drive/folders/1AZuXU0aAEyzaA6OQbI_5F74Q568EoY-d

2. Actualiza con cuidado el **día** 📅
3. El número de **corte diario** 🔢  
4. La cosecha que estás asignando.

In [ ]:
from datetime import datetime, timedelta

id_drive_pedidos = '1yVMEVT9zooZXOsiYHnZ1FZVGDZJQQ-sv'
id_drive_ctes = '1UQvGtFjCLp47JrP7vuBRQpg9cfSAlJxH'
id_drive_hist = '1zvW-Dxow9gz1Dnbpg_jO7my4wadDvJDW'
id_drive_salidas = '1AZuXU0aAEyzaA6OQbI_5F74Q568EoY-d'
id_sheets_tc2 = '1k8rguLeF1O33XCaVDxPiQ1C4SbxLDSIeqNcriYtsF-k'

fh_salida = '2026-02-19' #@param{type:'date'}
fh_salida_dt = datetime.strptime(fh_salida, '%Y-%m-%d')
dia_salida = str(fh_salida_dt.day).zfill(2)
mes_salida = str(fh_salida_dt.month).zfill(2)
anio_salida = str(fh_salida_dt.year).zfill(4)
corte = '3' #@param{type:'string'}[1,2,3]
fh_de_asignacion = fh_salida_dt.strftime('%d-%m-%Y')

cosecha = 'Cosecha Feb 26' #@param{type:'string'}

actualizar_tc = 'S' #@param{type:'string'}['S','N']

nb_carpeta_ctes_mes = f'{anio_salida}{mes_salida}'
nb_ctes_csv = f'report_{anio_salida}{mes_salida}{dia_salida}_c{corte}.csv'
nb_sheet_salida = f'Salidas {anio_salida}{mes_salida}{dia_salida}_c{corte}'

In [ ]:
dia_salida, mes_salida, anio_salida, corte, nb_carpeta_ctes_mes, nb_ctes_csv, cosecha

## Paqueterías

In [ ]:
import os

from_drive = True  # same flag you use everywhere

if os.environ.get("HERMES_BOOTSTRAPPED") != "1":
    # ---------- GIT ON COLAB ONLY ----------
    try:
        from google.colab import userdata

        git_token = userdata.get('gitToken')
        git_user = userdata.get('gitUser')
        git_url = f'https://{git_token}@github.com/rene-aum/Hermes.git'
        branch_to_pull = 'dev'

        os.chdir('/content')

        if not os.path.isdir('Hermes'):
            !git clone {git_url}

        %cd Hermes
        !git fetch origin {branch_to_pull}
        !git checkout {branch_to_pull}
        !git pull origin {branch_to_pull}

        !pip install -r utils/src/requirements.txt
        %cd Asignacion

    except Exception as e:
        print(e)
        print('Running in other environment not colab probably!')

    # ---------- DRIVE + SHEETS ----------
    if from_drive:
        from pydrive2.auth import GoogleAuth
        from pydrive2.drive import GoogleDrive
        from google.colab import auth
        from oauth2client.client import GoogleCredentials
        import gspread
        from google.auth import default
        from gspread_dataframe import set_with_dataframe
        import gdown

        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        drive = GoogleDrive(gauth)

        creds, _ = default()
        gc = gspread.authorize(creds)

    os.environ["HERMES_BOOTSTRAPPED"] = "1"
else:
    print("Bootstrap already done, assuming orchestrator ran it.")

In [ ]:
import sys
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
import sys
sys.path.append('..')
sys.path.append('../..')
from utils.utils import (get_dates_dataframe,
                       add_year_week,
                       custom_read,
                       process_columns,
                       remove_accents)

from utils.drive_toolbox import(from_drive_to_local,
                             get_last_modification_date_drive,
                             create_sheets_in_drive_folder,
                             update_sheets_in_drive_folder,
                             read_from_google_sheets,
                             list_file_ids_for_drive_folder,
                             create_csv_file_in_drive_folder,
                             write_csv_to_drive,
                             read_csv_from_drive,
                             append_dataframe_to_google_sheet_from_range)
from utils.src.constants import (atlas_consumo_output_folder_id,
                           consumo_sheets_ids_dict,
                           folder_id_bauto_gabo,
                           id_reporte_ventas,
                           id_edas_referenciados,
                           id_torre_de_control
                           )


warnings.filterwarnings('ignore')



In [ ]:
# EXTRAS

# --------- BUSCAR EN SUBCARPETAS -------------------------------------------
from googleapiclient.discovery import build
creds, _ = default()

servicedrive = build("drive", "v3", credentials=creds)
service_sheets = build("sheets", "v4", credentials=creds)

FOLDER_MIME = "application/vnd.google-apps.folder"

def listar_archivos(folder_id, mime_types=None):
    """
    folder_id: ID de la carpeta raíz
    mime_types: None | string | lista de strings
    """
    if isinstance(mime_types, str):
        mime_types = [mime_types]

    resultados = {}

    def recorrer(fid):
        page_token = None
        while True:
            resp = servicedrive.files().list(
                q=f"'{fid}' in parents and trashed = false",
                fields="nextPageToken, files(id, name, mimeType)",
                pageToken=page_token,
                supportsAllDrives=True,
                includeItemsFromAllDrives=True
            ).execute()

            for f in resp.get("files", []):
                if f["mimeType"] == FOLDER_MIME:
                    recorrer(f["id"])
                else:
                    if mime_types is None or f["mimeType"] in mime_types:
                        resultados[f["name"]] = f["id"]

            page_token = resp.get("nextPageToken")
            if not page_token:
                break

    recorrer(folder_id)
    return resultados


# -------------- LEER CON ENCODING ------------------------------------------
import io
import pandas as pd
from googleapiclient.http import MediaIoBaseDownload

def read_csv_from_drive_v3(drive_service, file_id, **read_csv_kwargs):
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()

    fh.seek(0)
    return pd.read_csv(fh, **read_csv_kwargs)


# import io
# def read_csv_from_drive2(drive, file_id, encoding="latin-1", **read_csv_kwargs):
#     f = drive.CreateFile({"id": file_id})
#     f.FetchContent()
#     b = f.content.getvalue()  # bytes
#     return pd.read_csv(io.BytesIO(b), encoding=encoding, **read_csv_kwargs)

# ------------- Todas las columnas ------------------------------------------
pd.set_option('display.max_columns', 100)

# ------------- IMPRIMIR CON COLORES ----------------------------------------
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[94m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
import math
from zoneinfo import ZoneInfo

def borrar_hojas(spreadsheet_id, nb_hojas):
    spreadsheet = service_sheets.spreadsheets().get(
        spreadsheetId=spreadsheet_id
    ).execute()

    sheet_ids = []
    for sheet in spreadsheet["sheets"]:
        if sheet["properties"]["title"] in nb_hojas:
            sheet_ids.append(sheet["properties"]["sheetId"])

    request = {
        "requests": [
            {"deleteSheet": {"sheetId": s_id}}
            for s_id in sheet_ids
            ]
    }

    service_sheets.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body=request
    ).execute()
    print(f"{nb_hojas} eliminada(s)")
    return

    print("Hoja no encontrada")

def crear_hojas_sheets(spreadsheet_id, nb_hojas, quitar_cuadricula = True, fila_congelada = 1):
  """
  nb_hojas: lista con los nombres de las hojas a crear
  quitar_cuadricula: True | False es para hacer invisibles los bordes/cuadrícula de las celdas
  """

  request = {
      "requests": [
          {"addSheet": {"properties": {"title": nombre,
                                       'gridProperties': {'hideGridlines':quitar_cuadricula,
                                                          'frozenRowCount':fila_congelada}
                                       }
                        }
           }
          for nombre in nb_hojas
      ]
  }

  service_sheets.spreadsheets().batchUpdate(
      spreadsheetId = id_sheets_salida,
      body=request
  ).execute()
  print(f'{nb_hojas} creadas')


def formato_hojas_sheets(sheets_id, nb_hojas, n_columnas, tamanio_letra = 11, letra = 'Source Serif 4', rgb_encabezado = [0.1, 0.3, 0.7], ):
  spreadsheet = service_sheets.spreadsheets().get(
      spreadsheetId=sheets_id
  ).execute()

  sheet_ids = []
  for sheet in spreadsheet["sheets"]:
      if sheet["properties"]["title"] in nb_hojas:
          sheet_ids.append(sheet["properties"]["sheetId"])

  # requests de formato
  requests = []
  for sh_id in sheet_ids:
    # A) Fuente para TODA la hoja
    r_global = {
        "repeatCell": {
            "range": {
                "sheetId": sh_id
            },
            "cell": {
                "userEnteredFormat": {
                    "textFormat": {
                        "fontFamily": letra,
                        "fontSize": tamanio_letra
                    }
                }
            },
            "fields": "userEnteredFormat.textFormat(fontFamily,fontSize)"
        }
    },

    # B) Color solo en encabezado (fila 1)
    r_encabezado = {
        "repeatCell": {
            "range": {
                "sheetId": sh_id,
                "startRowIndex": 0,
                "endRowIndex": 1,
                'startColumnIndex':0,
                'endColumnIndex':n_columnas
            },
            "cell": {
                "userEnteredFormat": {
                    "backgroundColor": {
                        "red": rgb_encabezado[0],
                        "green": rgb_encabezado[1],
                        "blue": rgb_encabezado[2]
                    },
                    "textFormat": {
                        "bold": True,
                        "foregroundColor": {
                            "red": 1,
                            "green": 1,
                            "blue": 1
                        }
                    }
                }
            },
            "fields": "userEnteredFormat(backgroundColor,textFormat.bold,textFormat.foregroundColor)"
        }
    }
    r_anchoColumnas = {
        "autoResizeDimensions": {
            "dimensions": {
                "sheetId": sh_id,
                "dimension": "COLUMNS",
                "startIndex": 0,
                "endIndex": 20   # ajusta primeras 20 columnas
            }
        }
    }
    requests.append(r_global)
    requests.append(r_encabezado)
    requests.append(r_anchoColumnas)

  service_sheets.spreadsheets().batchUpdate(
      spreadsheetId=sheets_id,
      body={"requests": requests}
      ).execute()

## Leemos base de pedidos y nos quedamos con los nuevos desde el último corte

In [ ]:
dict_PedidosSalesForce_id = listar_archivos(id_drive_pedidos)
dict_PedidosSalesForce_id = {
    x.split('_')[1]: v
    for x, v in dict_PedidosSalesForce_id.items()
}

fto_fh = '%Y-%m-%d-%H'
fhs_pedidos = list(dict_PedidosSalesForce_id.keys())
fhs_pedidos = sorted([datetime.strptime(x, fto_fh) for x in fhs_pedidos], reverse=True)

In [ ]:
ults_pdds = fhs_pedidos[0].strftime(fto_fh)
pens_pdds = fhs_pedidos[1].strftime(fto_fh)
print(f'Vamos a cargar pedidos entre {color.DARKCYAN} {fhs_pedidos[0]} y {fhs_pedidos[1]} {color.END} (ignorando minutos)')

In [ ]:
pdds_sf_hoy = read_csv_from_drive_v3(servicedrive, dict_PedidosSalesForce_id[ults_pdds],encoding = 'latin1')
pdds_sf_ayer = read_csv_from_drive_v3(servicedrive, dict_PedidosSalesForce_id[pens_pdds],encoding = 'latin1')
columnas_sf = ['num_pedido','id_pedido','id_cuenta','nb_cuenta','id_vendedor','nb_comprador',
                  'id_comprador','estatus','precio','fh_creacion','anticipo','estatus2','nb_producto','niv','desc','espacio','sku']
pdds_sf_hoy.columns = pdds_sf_hoy.iloc[7].reset_index(drop=True)
pdds_sf_hoy = pdds_sf_hoy.iloc[8:].reset_index(drop=True)
pdds_sf_hoy = pdds_sf_hoy.dropna(axis=1, how="all")
pdds_sf_hoy.drop(columns= np.nan, inplace = True)
if pdds_sf_hoy.shape[1]!=len(columnas_sf):  # 11 Feb 2026 agregamos columna SKU en el reporte de pedidos. Los previos no lo traerán, por lo que agregamos columna a mano
  pdds_sf_hoy['sku'] = 'no_disponible'
pdds_sf_hoy.columns = columnas_sf
i_total = list(pdds_sf_hoy[pdds_sf_hoy['num_pedido']=='Total'].index)[0]
pdds_sf_hoy = pdds_sf_hoy.iloc[:i_total]
pdds_sf_hoy = pdds_sf_hoy[pd.to_datetime(pdds_sf_hoy['fh_creacion']) >= datetime(2025,7,1)].sort_values(by='num_pedido').reset_index(drop=True)

pdds_sf_ayer.columns = pdds_sf_ayer.iloc[7].reset_index(drop=True)
pdds_sf_ayer = pdds_sf_ayer.iloc[8:].reset_index(drop=True)
pdds_sf_ayer = pdds_sf_ayer.dropna(axis=1, how="all")
pdds_sf_ayer.drop(columns= np.nan, inplace = True)
if pdds_sf_ayer.shape[1]!=len(columnas_sf):  # 11 Feb 2026 agregamos columna SKU. Los previos no lo traerán, por lo que agregamos columna a mano
  pdds_sf_ayer['sku'] = 'no_disponible'
pdds_sf_ayer.columns = columnas_sf
i_total = list(pdds_sf_ayer[pdds_sf_ayer['num_pedido']=='Total'].index)[0]
pdds_sf_ayer = pdds_sf_ayer.iloc[:i_total]
pdds_sf_ayer = pdds_sf_ayer[pd.to_datetime(pdds_sf_ayer['fh_creacion']) >= datetime(2025,7,1)].sort_values(by='num_pedido').reset_index(drop=True)

pdds_sf = pdds_sf_hoy[~pdds_sf_hoy['num_pedido'].isin(pdds_sf_ayer['num_pedido'])].copy().reset_index(drop=True)

display(pdds_sf.sample())
print(f'{color.BOLD}{color.CYAN}Tenemos {pdds_sf.shape[0]} pedidos nuevos{color.END}')

In [ ]:
# CELDA DE VALIDACIÓN
fh_final_pdds, fh_inicial_pdds = fhs_pedidos[0].replace(hour=0), fhs_pedidos[1].replace(hour=0)
fhs_creacion_ls = [datetime.strptime(x, '%d/%m/%Y') for x in pdds_sf['fh_creacion'].unique().tolist()]

if not all([x <= fh_final_pdds and x >= fh_inicial_pdds for x in fhs_creacion_ls]):
    print(f"{color.RED}Hay pedidos con fechas fuera del rango de actualización{color.END}")
    raise SystemExit

## Pegamos datos de clientes

In [ ]:
id_drive_ctes_mes = list_file_ids_for_drive_folder(drive, id_drive_ctes)[f'{nb_carpeta_ctes_mes}']
archs_ctesnvos = list_file_ids_for_drive_folder(drive, id_drive_ctes_mes)

ctes_nvos = read_csv_from_drive_v3(servicedrive, archs_ctesnvos[nb_ctes_csv], encoding = 'latin-1')
print(f'{color.BOLD}{color.CYAN}Se cargó base {nb_ctes_csv}{color.END}')
# ctes_nvos['fh_actualizacion'] = arch_reciente[0]
ctes_nvos.rename(columns = {'Id Comercio Externo':'id_am', 'Teléfono':'phone', 'Email':'email'}, inplace=True)

ctes_nvos = ctes_nvos[~ctes_nvos['id_am'].isna()]
ctes_nvos['id_am'] = ctes_nvos['id_am'].astype(int)
ctes_nvos = ctes_nvos[['id_am','phone','email']]
ctes_nvos['origen_arch'] = 'ctes_sf'
ctes_nvos.sample()

In [ ]:
# Aqui debo dar prioridad a la base AcClientes sobre la de andrés, porque hay registros que ahí están más completos (comentario de Rene)
ctes = read_from_google_sheets(gc, consumo_sheets_ids_dict['AcClientes'])
ctes['origen_arch'] = 'ctes_ac'

ctes = pd.concat([ctes, ctes_nvos])
ctes = ctes.drop_duplicates('id_am',keep='first').reset_index(drop=True)
ctes['phone'] = pd.to_numeric(ctes['phone'].fillna(0),errors='coerce').astype('Int64').astype(str).apply(lambda x: x[-10:])
ctes['id_am'] = ctes['id_am'].astype(int)

display(ctes.sample())
print(f'{color.BOLD}{color.CYAN}Tenemos {ctes.shape[0]} clientes{color.END}')

In [ ]:
# Pegamos info de clientes a los nuevos pedidos
pdds_sf['id_comprador'] = pdds_sf['id_comprador'].astype(int)
pdds_sf1 = pdds_sf.merge(ctes[['id_am','email','phone','origen_arch']], how = 'left', left_on = 'id_comprador', right_on = 'id_am')
pdds_sf1.sample(1)

In [ ]:
if not pdds_sf1[(pdds_sf1['email'].isna()) | (pdds_sf1['phone'].isna())].shape[0] == 0:
    print(f"{color.RED}Hay pedidos sin datos de comprador{color.END}")
    raise SystemExit

## Buscamos datos de pedidos nuevos en histórico y nos quedamos con los que cumplan definición de leads nuevos

In [ ]:
id_hist = '1zvW-Dxow9gz1Dnbpg_jO7my4wadDvJDW'
hist = list_file_ids_for_drive_folder(drive, id_hist)
hist = [v for i,v in hist.items() if '_latest.csv' in i ][0]
hist = read_csv_from_drive_v3(servicedrive, hist, encoding='latin-1' )

hist['fecha de asignacion'] = pd.to_datetime(hist['fecha de asignacion'], format = '%Y-%m-%d')
hist = hist.sort_values(by='fecha de asignacion', ascending=False)
hist.sample(1)

In [ ]:
hist_id = hist[['id comprador','id lead','estatus de lead']].copy().drop_duplicates('id comprador').dropna(subset='id comprador')
hist_mail = hist[['mail comprador','id lead','estatus de lead']].copy().drop_duplicates('mail comprador').dropna(subset='mail comprador')
hist_fon = hist[['telefono comprador','id lead','estatus de lead']].copy().drop_duplicates('telefono comprador').dropna(subset='telefono comprador')

hist_id.columns = ['id_comprador','id_lead','estatus_lead']
hist_mail.columns = ['email','id_lead','estatus_lead']
hist_fon.columns = ['phone','id_lead','estatus_lead']

pdds_sf1['id_comprador'] = pdds_sf1['id_comprador'].astype(str)
pdds_sf1['phone'] = pd.to_numeric(pdds_sf1['phone'].astype(str).str[-10:],errors = 'coerce').astype('Int64')
hist_fon['phone'] = pd.to_numeric(hist_fon['phone'].astype(str).str[-10:],errors='coerce').astype('Int64')
pdds_sf2 = pdds_sf1.merge(hist_id[['id_comprador','id_lead','estatus_lead']], how = 'left', on = 'id_comprador', suffixes = ['','_conid'])
pdds_sf2 = pdds_sf2.merge(hist_mail[['email','id_lead','estatus_lead']], how = 'left', on = 'email', suffixes = ['','_conemail'])
pdds_sf2 = pdds_sf2.merge(hist_fon[['phone','id_lead','estatus_lead']], how = 'left', on = 'phone', suffixes = ['','_confon'])

pdds_sf2['id_lead'] = np.where(pdds_sf2['id_lead'].notna(), pdds_sf2['id_lead'],
                               np.where(pdds_sf2['id_lead_conemail'].notna(), pdds_sf2['id_lead_conemail'],
                                        np.where(pdds_sf2['id_lead_confon'].notna(), pdds_sf2['id_lead_confon'],np.nan)))

total_congruentes_incongruentes = len(pdds_sf2)
cerrados = ['COMPRA EXITOSA ','COMPRA EXITOSA','CERRADO','NA']
pdds_sf2['aux'] =((~pdds_sf2['estatus_lead'].fillna('NA').isin(cerrados) )*1 + (~pdds_sf2['estatus_lead_conemail'].fillna('NA').isin(cerrados))*1 +
                  (~pdds_sf2['estatus_lead_confon'].fillna('NA').isin(cerrados))*1)
pdds_stts_incongruente = pdds_sf2[pdds_sf2['aux'].between(1,2)].copy()
pdds_sf2 = pdds_sf2[pdds_sf2['aux'].isin([0,3])].copy()

pdds_sf2['estatus_lead'] = np.where(pdds_sf2['estatus_lead'].notna(), pdds_sf2['estatus_lead'],
                               np.where(pdds_sf2['estatus_lead_conemail'].notna(), pdds_sf2['estatus_lead_conemail'],
                                        np.where(pdds_sf2['estatus_lead_confon'].notna(), pdds_sf2['estatus_lead_confon'],np.nan)))

nvos_leads = ( (pdds_sf2['id_lead'].isna()) | (pdds_sf2['estatus_lead'].isin(cerrados) ) )

leads_ok = pdds_sf2[~nvos_leads].copy()
leads_ok = pd.concat([leads_ok, pdds_stts_incongruente])

# leads_nvos = pdds_sf2.copy()
# leads_nvos = leads_nvos[~leads_nvos['num_pedido'].isin(pedidos_multicontacto['num_pedido'].unique())]
leads_nvos = pdds_sf2[nvos_leads].copy()

print(len(leads_ok), len(leads_nvos), total_congruentes_incongruentes)
display(pdds_stts_incongruente)

In [ ]:
if not len(leads_ok) + len(leads_nvos) == total_congruentes_incongruentes:
    print(f"{color.RED}El total de pedidos que requieren leads, considerando los incongruentes, no cuadra con el total de pedidos iniciales{color.END}")
    print(f"{color.BOLD}{color.CYAN}La clasificación de leads ok (pedidos que no requieren lead nuevo) y leads nuevos está perdiendo alguno(s) de los pedidos con los que iniciamos {color.END}")
    raise SystemExit

In [ ]:
leads_nvos = leads_nvos[['id_comprador','espacio','phone','email','nb_comprador']].drop_duplicates('id_comprador', keep='last').reset_index(drop=True)
leads_nvos['espacio'] = leads_nvos['espacio'].replace({'Metrópoli Patriotismo':'patriotismo','Samara Satélite':'samara','Reforma 510':'torre'})
print(f'{color.BOLD}{color.CYAN} Tenemos {color.UNDERLINE}{len(leads_nvos)} leads nuevos{color.END}')

## Asignaciones

In [ ]:
# Leemos catálogo de asesores
id_asesores = '1xQkepXoIoEHNgdYUaLAT7FtDnuzYo1qb'
cat_as = list_file_ids_for_drive_folder(drive, id_asesores)['AsesoresEspacio']
centros = ['torre','samara','patriotismo','celula_credito']
assrs_actvs = {}
for c in centros:
  df = read_from_google_sheets(gc,cat_as,c)
  df['espacio'] = c
  df = df[df.activo==1].drop_duplicates(['asesor','espacio'])
  assrs_actvs[c] = df
assrs_actvs = pd.concat(assrs_actvs.values())
assrs_actvs = assrs_actvs.reset_index(drop=True)
assrs_actvs['asesor'] = assrs_actvs['asesor'].str.lower().str.strip()

# Número de asesores activos por espacio
num_assrs = {c: len(assrs_actvs[assrs_actvs.espacio==c]) for c in assrs_actvs.espacio.unique()}
num_assrs

In [ ]:
# Leads activos por asesor
dicc_espacios = {'MetrÃ³poli Patriotismo':'patriotismo','Samara SatÃ©lite':'samara','Reforma 510':'torre'}

leads_asesor_espacio = hist[~hist['estatus de lead'].isin(['COMPRA EXITOSA ','COMPRA EXITOSA', 'CERRADO'])].groupby(['asesor espacio','espacio automarket']).agg({'id lead':'nunique'}).reset_index()
leads_asesor_espacio.columns = ['asesor','espacio','leads']
leads_asesor_espacio['asesor'] = leads_asesor_espacio['asesor'].str.lower().str.strip()
leads_asesor_espacio['espacio'] = leads_asesor_espacio['espacio'].replace(dicc_espacios)
leads_asesor_espacio = leads_asesor_espacio[~leads_asesor_espacio['asesor'].str.strip().isin(['PRUEBA','prueba','#N/A()','#n/a ()'])]
leads_asesor_espacio = leads_asesor_espacio.groupby(['asesor','espacio']).agg({'leads':'sum'}).reset_index()


leads_asesor_cred = hist[~hist['estatus de lead'].isin(['COMPRA EXITOSA ','COMPRA EXITOSA', 'CERRADO'])].groupby(['asesor credito']).agg({'id lead':'nunique'}).reset_index()
leads_asesor_cred.columns = ['asesor','leads']
leads_asesor_cred['asesor'] = leads_asesor_cred['asesor'].str.lower().str.strip()
leads_asesor_cred['espacio'] = 'celula_credito'
leads_asesor_cred = leads_asesor_cred[~leads_asesor_cred['asesor'].str.strip().isin(['PRUEBA', 'prueba', '#N/A()', '#n/a ()', '', ' '])]
leads_asesor_cred = leads_asesor_cred.groupby(['asesor','espacio']).agg({'leads':'sum'}).reset_index()

leads_asesor = pd.concat([leads_asesor_espacio, leads_asesor_cred])

In [ ]:
assrs_actvs_leads = assrs_actvs.merge(leads_asesor, how = 'left', on = ['asesor','espacio'])
assrs_actvs_leads = assrs_actvs_leads.sort_values(['espacio','leads']).reset_index(drop=True)

assrs_actvs_leads['llave'] = assrs_actvs_leads.groupby('espacio').cumcount()
assrs_actvs_leads['leads'] = assrs_actvs_leads['leads'].fillna(0)

leads_asesor_cred = assrs_actvs_leads[assrs_actvs_leads['espacio'] == 'celula_credito'].rename(columns = {'asesor':'asesor credito'})
leads_asesor_esp = assrs_actvs_leads[assrs_actvs_leads['espacio'] != 'celula_credito'].rename(columns = {'asesor':'asesor espacio'})
display(leads_asesor_cred)
display(leads_asesor_esp)

In [ ]:
# Vamos a procesar distinto los leads que ya tuvieron gestión en algún espacio. Los seleccionamos con inner merge vs histórico
leads_nvos_comprprevio = leads_nvos.copy().rename(columns = {'espacio':'espacio sf'}) #espacio sf es el nuevo espacio que asigna salesforce
leads_nvos_comprprevio = leads_nvos_comprprevio.merge(
    hist[['id comprador','espacio automarket','asesor espacio']].rename(
    columns = {'espacio automarket':'espacio previo', 'asesor espacio':'asesor espacio previo'}
    ).drop_duplicates(
        'id comprador',keep='first'),
                                        how = 'inner', left_on = 'id_comprador', right_on = 'id comprador').drop(columns = ['id comprador'])

leads_nvos_comprprevio['espacio previo'] = leads_nvos_comprprevio['espacio previo'].replace(dicc_espacios)

# Mergeamos por el espacio que tenía en su lead anterior (espacio 1) con catálogo de asesores y nos quedamos con el primer asesor que cruce y no haya tenido antes (columna aux)
leads_nvos_comprprevio = leads_nvos_comprprevio.merge(leads_asesor_esp.rename(columns = {'asesor espacio':'asesor espacio nvo'}), how = 'left', left_on = 'espacio previo', right_on = 'espacio')
leads_nvos_comprprevio['aux'] = (leads_nvos_comprprevio['asesor espacio previo'].str.lower().str.strip() == leads_nvos_comprprevio['asesor espacio nvo'].str.lower().str.strip())*1
leads_nvos_comprprevio = leads_nvos_comprprevio.sort_values(by='aux', ascending=True)
leads_nvos_comprprevio = leads_nvos_comprprevio.drop_duplicates(['id_comprador'], keep = 'first').reset_index(drop=True)
leads_nvos_comprprevio = leads_nvos_comprprevio.drop(columns = ['espacio sf','espacio previo','asesor espacio previo','activo','leads','llave','aux']).rename(columns = {'asesor espacio nvo':'asesor espacio'})
display(leads_nvos_comprprevio)

In [ ]:
leads_nvos = leads_nvos[~leads_nvos['id_comprador'].isin(leads_nvos_comprprevio['id_comprador'].unique())]
leads_nvos['llave_esp'] = leads_nvos.groupby('espacio').cumcount() % leads_nvos['espacio'].map(num_assrs)
leads_nvos = leads_nvos.merge(leads_asesor_esp, how='left', left_on = ['espacio','llave_esp'], right_on = ['espacio','llave'])

In [ ]:
salida_leads = pd.concat([leads_nvos, leads_nvos_comprprevio]).sort_values(by='id_comprador').reset_index(drop=True)

salida_leads['llave_celcred'] = salida_leads.index % num_assrs['celula_credito']
salida_leads = salida_leads.merge(leads_asesor_cred, how = 'left', left_on = 'llave_celcred', right_on = 'llave', suffixes = ['','_cred'])
leads_nvos

In [ ]:
hist['conteo_leads'] = hist['id lead'].str.replace('|','0').str[-6:].astype(int)
ultimo_lead = hist['conteo_leads'].max()
ultimo_lead

In [ ]:
salida_leads = salida_leads.reset_index(drop=True)
salida_leads = salida_leads.reset_index()
salida_leads['index'] = salida_leads['index']+1
salida_leads['id lead'] = salida_leads['index'] + ultimo_lead
salida_leads['id lead'] = 'LAA-' + salida_leads['id lead'].astype(str).str.zfill(6)
salida_leads.drop(columns = ['llave','index','activo','leads'], inplace = True)
salida_leads['folio bauto tc'] = 'x'
salida_leads['origen automarket'] = 'Apartado'
salida_leads['cosecha'] = cosecha
salida_leads['fecha de asignacion'] = fh_de_asignacion

salida_leads.rename(columns = {'id_comprador':'id comprador','espacio':'espacio automarket','phone':'telefono comprador','email':'mail comprador',
                               'nb_comprador':'nombre comprador'},inplace=True)
salida_leads = salida_leads[['id lead','origen automarket','cosecha','id comprador','folio bauto tc',
                             'nombre comprador','mail comprador','telefono comprador','asesor credito','espacio automarket','asesor espacio','fecha de asignacion']]
salida_leads

In [ ]:
id_l_nvos = salida_leads['id lead'].unique()

if not hist[hist['id lead'].isin(id_l_nvos)].shape[0] == 0:
    print(f"{color.RED}Se están duplicando id leads respecto a los que ya existían en la torre de control{color.END}")
    raise SystemExit

In [ ]:
asgns_as_es = salida_leads.groupby(['asesor espacio','espacio automarket']).agg(nuevos = ('id comprador','nunique')).reset_index()
asgns_as_cr = salida_leads.groupby(['asesor credito']).agg(nuevos = ('id comprador','nunique')).reset_index().rename(columns = {'asesor credito':'asesor espacio'})
asgns_as_cr['espacio automarket'] = 'celula_credito'

asgns_as = pd.concat([asgns_as_es, asgns_as_cr])
asgns_as['asesor espacio'] = asgns_as['asesor espacio'].str.lower()
asgns_as['espacio automarket'] = asgns_as['espacio automarket'].replace({'Metrópoli Patriotismo':'patriotismo','Reforma 510':'torre','Samara Satélite':'samara'})

assrs_actvs_leads.columns = ['asesor espacio','activo','espacio automarket','leads','llave']
asgns_res = assrs_actvs_leads.merge(asgns_as, how = 'left', on = ['asesor espacio','espacio automarket'])

asgns_res['leads_nuevos'] = asgns_res['leads'] + asgns_res['nuevos']
asgns_res

## Contact Center

In [ ]:
id_comprador_leadsnvos = salida_leads['id comprador'].unique()
leads_cc = hist[hist['estatus de lead']=='Contact Center']['id comprador'].unique()

cc = pdds_sf2[pdds_sf2['id_comprador'].isin(list(id_comprador_leadsnvos) + list(leads_cc))].reset_index(drop=True)
cc = cc[['num_pedido','id_comprador','id_vendedor','fh_creacion']]
datos_nvosleads_previoscc = pd.concat([salida_leads,hist]).reset_index(drop=True)  #.drop_duplicates('id')
cols = ['id lead','id comprador','fecha de asignacion','origen automarket','asesor espacio','espacio automarket']
datos_nvosleads_previoscc = datos_nvosleads_previoscc[cols].drop_duplicates('id comprador')           # Deduplico y priorizo los que estamos asignando primero y después los históricos empezando por los más recientes (hist está ordenado así)
salida_cc = cc.merge(datos_nvosleads_previoscc, how='left',left_on='id_comprador',right_on = 'id comprador')
salida_cc['fecha de asignacion'] = salida_cc['fecha de asignacion'].astype(str).str[:10]
salida_cc['num_pedido'] = salida_cc['num_pedido'].astype(int)
salida_cc
# FALTA CRUZAR CONTRA EL HISTÓRICO PARA LA INFO DE LOS QUE YA ESTABAN PERO CON ESTATUS DE CONTACT CENTER

In [ ]:
salida_cc['id_vendedor'] = salida_cc['id_vendedor'].astype(str)
ctes['id_am'] = ctes['id_am'].astype(str)

salida_cc = salida_cc.merge(ctes[['id_am','billing_firstname','email','phone']], how = 'left', left_on = 'id_vendedor', right_on = 'id_am')

salida_cc['fecha de asignacion cc'] = datetime.today().strftime('%Y-%m-%d')
salida_cc = salida_cc[['id lead','fecha de asignacion','fecha de asignacion cc','origen automarket','asesor espacio','espacio automarket',
                       'id_comprador','num_pedido','fh_creacion','id_vendedor','billing_firstname','email','phone']]

## Guardado

In [ ]:
dicc_espacios2 = {'MetrÃ³poli Patriotismo': 'Metrópoli Patriotismo','Samara SatÃ©lite': 'Samara Satélite'}

salida_leads['espacio automarket'] = salida_leads['espacio automarket'].replace({'torre':'Reforma 510','patriotismo':'Metrópoli Patriotismo','samara':'Samara Satélite'})
salida_cc['espacio automarket'] = salida_cc['espacio automarket'].replace(dicc_espacios2).replace({'torre':'Reforma 510','patriotismo':'Metrópoli Patriotismo','samara':'Samara Satélite'})

salida_leads['asesor credito'] = salida_leads['asesor credito'].str.title()
salida_leads['asesor espacio'] = salida_leads['asesor espacio'].str.title()
salida_cc['asesor espacio'] = salida_cc['asesor espacio'].str.title()

salida_leads['estatus de lead'] = 'Contact Center'

In [ ]:
# Creamos la hoja de sheets de cero

ahora_dt = datetime.now(ZoneInfo("America/Mexico_City")).strftime('%d-%m-%Y %H:%M')
nb_sheets_salida = f'Salida {ahora_dt}'
nb_hojas = ['asignacion_apartados','salida_cc']

id_folder_mes_salida = list_file_ids_for_drive_folder(drive, id_drive_salidas)[f'{anio_salida}{mes_salida}']
create_sheets_in_drive_folder(gc, nb_sheets_salida, id_folder_mes_salida)
id_sheets_salida = listar_archivos(id_folder_mes_salida, mime_types='application/vnd.google-apps.spreadsheet')[nb_sheets_salida]

# Creamos hojas y eliminamos la hoja 1.
# Si esto da error, muy probablemente es porque ya hay un archivo con el mismo nombre en la carpeta. -----> Revisa la carpeta de drive.
crear_hojas_sheets(id_sheets_salida, nb_hojas)
borrar_hojas(id_sheets_salida,['Hoja 1'])

In [ ]:
# Guardamos la salida del proceso de asignación en hojas de respaldo

hoja_df = {'asignacion_apartados': salida_leads, 'salida_cc':salida_cc}
for hoja, df in hoja_df.items():
  update_sheets_in_drive_folder(gc, id_sheets_salida, hoja, df)
  formato_hojas_sheets(id_sheets_salida, [hoja], n_columnas = df.shape[1], letra = 'Source Serif 4' )

print(f'Las salidas se generaron y se guardaron en https://docs.google.com/spreadsheets/d/{id_sheets_salida}')

In [ ]:
# Tomamos fotos a torre de control y creamos hojas correspondientes para depositarlas en reporte de asignacion
ahora_dt = datetime.now(ZoneInfo("America/Mexico_City")).strftime('%d-%m-%Y %H:%M')

nb_foto_as = f'FotoAsig_TC_{ahora_dt}'
asig_foto = read_from_google_sheets(gc, id_sheets_tc2, sheetname='asignacion')

nb_foto_cc = f'FotoCC_TC_{ahora_dt}'
cc_foto = read_from_google_sheets(gc, id_sheets_tc2, sheetname='contact center')

crear_hojas_sheets(id_sheets_salida, [nb_foto_as, nb_foto_cc])

# ========================Asignaciones ==========================
# guardamos respaldo
update_sheets_in_drive_folder(gc, id_sheets_salida, nb_foto_as, asig_foto)
formato_hojas_sheets(id_sheets_salida, [nb_foto_as], n_columnas = asig_foto.shape[1], letra = 'Source Serif 4' )
# actualizamos si aplica
if actualizar_tc == 'S':
  append_dataframe_to_google_sheet_from_range(gc, id_sheets_tc2, 'asignacion', salida_leads)
else:
  print('No se actualizó la hoja de asignación en torre de control')
# ===============================================================

# ========================Contact Center ==========================
# guardamos respaldo
update_sheets_in_drive_folder(gc, id_sheets_salida, nb_foto_cc, cc_foto)
formato_hojas_sheets(id_sheets_salida, [nb_foto_cc], n_columnas = cc_foto.shape[1], letra = 'Source Serif 4' )
# actualizamos si aplica
if actualizar_tc == 'S':
  append_dataframe_to_google_sheet_from_range(gc, id_sheets_tc2, 'contact center', salida_cc)
else:
  print('No se actualizó la hoja de contact center en torre de control')
# ===============================================================